In [1]:
import pickle
import geopy
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px

In [2]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
#this user_agent was found in a web example, might need to be updated in the future if there's API request issues

def reverse_geocode(coord):
    '''
    Feed the lat and lon values into the reverse geocode API and get a zipcode back out
    '''
    location = geolocator.reverse(coord, exactly_one=True)
    address = location.raw['address']
    try:
        zipcode = address['postcode']
    except:
        zipcode = ''
        #set to an empty string instead of NaN or error
    return zipcode

#print(reverse_geocode("47.470706, -99.704723"))
# ^^ uncomment to verify the api is working

In [3]:
df = pd.read_csv('https://data.cityofnewyork.us/api/views/kk4q-3rt2/rows.csv')
df.columns.str.strip()
df.head()

#reverse_geocode(df.latlon[0][1] + ', ' + df.latlon[0][0])
# ^^ uncomment this to run a sample reverse geocode on the first entry in the df

,URL,OBJECTID,NAME,the_geom,LINE,NOTES
0,http://web.mta.info/nyct/service/,1,Astor Pl,POINT (-73.99106999861966 40.73005400028978),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s..."
1,http://web.mta.info/nyct/service/,2,Canal St,POINT (-74.00019299927328 40.71880300107709),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s..."
2,http://web.mta.info/nyct/service/,3,50th St,POINT (-73.98384899986625 40.76172799961419),1-2,"1-all times, 2-nights"
3,http://web.mta.info/nyct/service/,4,Bergen St,POINT (-73.97499915116808 40.68086213682956),2-3-4,"4-nights, 3-all other times, 2-all times"
4,http://web.mta.info/nyct/service/,5,Pennsylvania Ave,POINT (-73.89488591154061 40.66471445143568),3-4,"4-nights, 3-all other times"


In [4]:
df.drop(['URL','NOTES'], axis=1, inplace=True)

df['latlon'] = df['the_geom'].str.lstrip('POINT (').str.rstrip(')').str.split()
df['zipcode'] = df.apply(lambda row: reverse_geocode(row['latlon'][1] + ', ' + row['latlon'][0]), axis = 1)
# ^^^ takes a while, pretty sure there's a requests/min limit for the API used in reverse_geocode()

df.sort_values('NAME').head(50)

,OBJECTID,NAME,the_geom,LINE,latlon,zipcode
158,159,103rd St,POINT (-73.96837899960818 40.799446000334825),1,"[-73.96837899960818, 40.799446000334825]",10025
160,161,103rd St,POINT (-73.96137008267617 40.796060739904526),A-B-C,"[-73.96137008267617, 40.796060739904526]",10025-4403
457,458,103rd St,POINT (-73.94747800152219 40.79060000008452),4-6-6 Express,"[-73.94747800152219, 40.79060000008452]",10029
188,189,103rd St - Corona Plaza,POINT (-73.86269999830412 40.749865000555545),7,"[-73.86269999830412, 40.749865000555545]",11368
97,98,104th St,POINT (-73.83768300060997 40.681711001091195),A-S,"[-73.83768300060997, 40.681711001091195]",11419
13,14,104th-102nd Sts,POINT (-73.84443500029684 40.69516599823373),J-Z,"[-73.84443500029684, 40.69516599823373]",11418
449,450,110th St,POINT (-73.94424999687163 40.795020000113105),4-6-6 Express,"[-73.94424999687163, 40.795020000113105]",10029
189,190,111th St,POINT (-73.85533399834884 40.75172999941711),7,"[-73.85533399834884, 40.75172999941711]",11368
82,83,111th St,POINT (-73.83216299845388 40.68433100001238),A-S,"[-73.83216299845388, 40.68433100001238]",11419
208,209,111th St,POINT (-73.83679338454697 40.697114810696476),J,"[-73.83679338454697, 40.697114810696476]",11418


In [5]:
zipcode_list = list(df['zipcode'])
zipcode_list_cleaned = [entry[:5] for entry in zipcode_list]
print(zipcode_list_cleaned[:15])

##NOTE TO SELF, FIGURE OUT HOW TO DO THIS WHOLE THING AS A LAMBDA FUNCTION INSTEAD OF GENERATING A SEPERATE LIST.

['10003', '10013', '10107', '11213', '11207', '10463', '10026', '11207', '11377', '11101', '10002', '11207', '11208', '11418', '11227']


In [6]:
df['zipcode'] = pd.Series(zipcode_list_cleaned)

---

Now that the list of subway stations along with the zipcodes has been generated we can import the dataframe which contains the top 25 picks from the turnstile dataset

---

In [9]:
top_25_df = pd.read_pickle('stations_daily_top25.pkl')
top_25_df

,C/A,UNIT,STATION,LINENAME,DATE,ENTRIES,EXITS,Previous_date,Previous_Entries,Daily_Entries,STATION_LINE
106,A013,R081,49 ST,NQRW,08/30/2020,152575761,307855750,08/29/2020,152572931.0,2830.0,49 ST NQRW
107,A013,R081,49 ST,NQRW,08/31/2020,152578932,307861117,08/30/2020,152575761.0,3171.0,49 ST NQRW
108,A013,R081,49 ST,NQRW,09/01/2020,152584490,307867719,08/31/2020,152578932.0,5558.0,49 ST NQRW
109,A013,R081,49 ST,NQRW,09/02/2020,152590227,307874563,09/01/2020,152584490.0,5737.0,49 ST NQRW
112,A013,R081,49 ST,NQRW,09/05/2020,152607649,307894332,09/04/2020,152602116.0,5533.0,49 ST NQRW
...,...,...,...,...,...,...,...,...,...,...,...
14143,R506,R276,VERNON-JACKSON,7,09/14/2020,44280125,903021,09/13/2020,44277229.0,2896.0,VERNON-JACKSON 7
14144,R506,R276,VERNON-JACKSON,7,09/15/2020,44284620,903670,09/14/2020,44280125.0,4495.0,VERNON-JACKSON 7
14145,R506,R276,VERNON-JACKSON,7,09/16/2020,44289536,904299,09/15/2020,44284620.0,4916.0,VERNON-JACKSON 7
14146,R506,R276,VERNON-JACKSON,7,09/17/2020,44294522,904956,09/16/2020,44289536.0,4986.0,VERNON-JACKSON 7


In [ ]:
df['LINENAME'] = df.apply(lambda row: row['LINE'].rstrip('Express').replace('-',''), axis = 1)
# formats the line names so they more closely match the format in the turnstile df
# entries include more lines, try to merge based on whether the str in the turnstile df is in df, not ==

In [ ]:
df['STATION'] = df.apply(lambda row: row['NAME'].upper(), axis = 1)
df.sort_values('STATION').head()

In [ ]:
top_25_df.sort_values('STATION').groupby(['STATION','LINENAME']).head(1).head(20)

In [ ]:
station_name_list = list(df.NAME)
station_name_list_cleaned = [entry.upper().replace(' AVE',' AV').replace('RD-','-').replace('RD ',' ').replace('ND-','-').replace('ND ',' ').replace('TH-','-').replace('TH ',' ').replace('1ST-','1-').replace('1ST ','1 ') for entry in station_name_list]
df['STATION'] = pd.Series(station_name_list_cleaned)

In [ ]:
df.sort_values('STATION').head(10)

In [ ]:
postal_codes = df[['STATION','LINENAME','zipcode', 'latlon']]
postal_codes = postal_codes.sort_values('STATION').reset_index(drop=True)

In [ ]:
lat_list = list(df['latlon'])
lat_list_cleaned = [entry[1] for entry in lat_list]
lon_list_cleaned = [entry[0] for entry in lat_list]
postal_codes['lat'] = lat_list_cleaned
postal_codes['lon'] = lon_list_cleaned
postal_codes.drop(columns=['latlon'],inplace=True)
postal_codes

In [ ]:
#from: https://stackoverflow.com/questions/41815079/pandas-merge-join-two-data-frames-on-multiple-columns
#new_df = pd.merge(A_df, B_df,  how='left', left_on=['A_c1','c2'], right_on = ['B_c1','c2'])

new_df = pd.merge(top_25_df, postal_codes, how='left', left_on=['STATION','LINENAME'], right_on=['STATION','LINENAME'])
new_df.groupby(['STATION','LINENAME']).sum().sort_values('STATION').head(100)
new_df.head(100)

In [ ]:
finished_df = new_df.drop_duplicates().reset_index()
finished_df

In [ ]:
#postal_codes.to_csv ('postal_codes.csv', index = False, header=True)

In [ ]:
#finished_df.to_csv ('finished_df.csv', index = False, header=True)